# Question_1 : Decoding Message


In [ ]:
import cv2

image = cv2.imread('for_watson.png')

alpha = 80 # Contrast control (1.0-3.0)
beta = 90 # Brightness control (0-100)

adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

cv2.imshow('original', image)
cv2.imshow('adjusted', adjusted)
cv2.waitKey()

# Question 2: Vanishing Points

In [ ]:
import cv2 as cv
import numpy as np
import math


def Vanishing_point(x1, y1, x2, y2, x3, y3, x4, y4):
   a1= y2-y1
   a2= y4-y3
   b1= x1-x2
   b2= x3-x4
   c1=(y1*(x2-x1))- (x1* (y2-y1))
   c2=(y3*(x4-x3))- (x3* (y4-y3))

   x= ((b1*c2) - (b2*c1))/((a1*b2)- (a2*b1))
   y= ((c1*a2) - (c2*a1))/((a1*b2)- (a2*b1))

   return x, y

VP1_x, VP1_y = Vanishing_point(332, 176, 324, 238, 303, 177, 302, 253)
VP2_x, VP2_y = Vanishing_point(303, 177, 390, 193, 302, 253, 233, 162)
VP3_x, VP3_y = Vanishing_point(303, 177, 421, 171, 302, 253, 325, 237)


print('\nVanishing Point 1 x and y coordinates', VP1_x, VP1_y)
print('\nVanishing Point 1 x and y coordinates', VP2_x, VP2_y)
print('\nVanishing Point 1 x and y coordinates', VP3_x, VP3_y)

u1 = VP1_x
v1 = VP1_y
u2 = VP2_x
v2 = VP2_y
u3 = VP3_x
v3 = VP3_y

A_matrix = np.matrix([[ u1 - u3, v1 - v3], [ u2 - u3, v2 - v3]])
print(A_matrix)
B_matrix = np.matrix([[ (u1 - u3) * u2 + (v1 - v3) * v2], [ (u2 - u3) * u1 + (v2 - v3) * v1]])


X_matrix = np.linalg.inv(A_matrix) * B_matrix
x0 = float(X_matrix[0]) # Extracting data from X as x0
y0 = float(X_matrix[1]) # Extracting data from X as y0

print('\nPrincipal Points  in pixel : \nX0 = ', x0, '\nY0 = ', y0)


# Finding the focal length



f = math.sqrt((-(u1 - x0) * (u2 - x0)) - ((v1 - y0) * (v2 - y0))) # Focal Point
print("focal length in pixel : ",f)


## Since each coordinates are measured in pixels, I am converting it into mm. For 610 pixels, it aquire 10.5 cm. Hence focal length in cm will be:

f_cm = f*(10.5/610)

print("Focal length in cm is : ",f_cm)

## Similarly, x0 and y0 in cm is
x0_cm = x0*(10.5/610)
y0_cm = y0*(10.5/610)

print('\nPrincipal Points in cm is : \nX0_cm = ', x0_cm, '\nY0_cm = ', y0_cm)

k_matrix = np.array([[1.42,0,5.39],[0,1.46,3.42],[0,0,1]])
print("k_matrix = \n ",k_matrix)



# Question 3 Aruco Camera Calibration


In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
# from skimage import data
# from skimage.color import rgb2hsv
# from skimage import io
import cv2
import cv2.aruco as aruco
import os

aruco_dict = aruco.Dictionary_get (aruco.DICT_6X6_1000)
# Dimensions in cm
marker_length = 3.7
marker_separation = 1.1
arucoParams = aruco.DetectorParameters_create()
board = aruco.GridBoard_create(5, 7, marker_length, marker_separation, aruco_dict)
imboard = board.draw( (2000, 2000))
cv2.imwrite("./pics/chessboard.png", imboard)
fig = plt.figure ()
ax = fig.add_subplot (1,1,1)
plt.imshow (imboard, interpolation = "nearest")
ax.axis ("off")
plt.show()

In [ ]:
counter, corners_list, id_list = [], [], []
first = 0
image = "/Users/pushkarshinde/Downloads/arukoTags/images"
# Find the ArUco markers inside each image
for filename in os.listdir(image):
    print(filename)
    img = cv2.imread(os.path.join(image, filename))
    if img is not None:
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        corners, ids, rejected = aruco.detectMarkers (img_gray, aruco_dict, parameters=arucoParams)
    if first == 0:
        corners_list = corners
        id_list = ids
    else:
        corners_list = np.vstack((corners_list, corners))
        id_list = np.vstack((id_list,ids))
    first = first + 1
    counter.append(len(ids))



In [ ]:
counter = np.array(counter)
# Actual calibration
ret, mtx, dist, rvecs, tvecs = aruco.calibrateCameraAruco(corners_list, id_list,counter,board,img_gray.shape,None,None)

print("full K matrix: \n",mtx)
print("distortion parameters are: \n",dist)


# Question 4: Augmenting Cube over Aruco 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
# from skimage import data
# from skimage.color import rgb2hsv
# from skimage import io
import cv2
import cv2.aruco as aruco
import os

aruco_dict = aruco.Dictionary_get (aruco.DICT_6X6_1000)
# Dimensions in cm
marker_length = 3.7
marker_separation = 1.1
arucoParams = aruco.DetectorParameters_create()
board = aruco.GridBoard_create(5, 7, marker_length, marker_separation, aruco_dict)
imboard = board.draw( (2000, 2000))
cv2.imwrite("./pics/chessboard.png", imboard)
fig = plt.figure ()
ax = fig.add_subplot (1,1,1)
plt.imshow (imboard, interpolation = "nearest")
ax.axis ("off")
plt.show()

In [ ]:
counter, corners_list, id_list = [], [], []
first = 0
image = "/Users/pushkarshinde/Downloads/arukoTags/images"
# Find the ArUco markers inside each image
for filename in os.listdir(image):
    print(filename)
    img = cv2.imread(os.path.join(image, filename))
    if img is not None:
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        corners, ids, rejected = aruco.detectMarkers (img_gray, aruco_dict, parameters=arucoParams)
    if first == 0:
        corners_list = corners
        id_list = ids
    else:
        corners_list = np.vstack((corners_list, corners))
        id_list = np.vstack((id_list,ids))
    first = first + 1
    counter.append(len(ids))



In [ ]:
counter = np.array(counter)
# Actual calibration
ret, mtx, dist, rvecs, tvecs = aruco.calibrateCameraAruco(corners_list, id_list,counter,board,img_gray.shape,None,None)

print("full K matrix: \n",mtx)
print("distortion parameters are: \n",dist)


In [ ]:
tag = cv2.imread('/Users/pushkarshinde/Downloads/arukoTags/imagesIMG_0781.jpg')

In [ ]:
plt.figure(figsize = (3,3))
plt.imshow(tag)
plt.show()

In [ ]:
arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_1000)
arucoParams = cv2.aruco.DetectorParameters_create()
(corners, ids, rejected) = cv2.aruco.detectMarkers(tag, arucoDict, parameters=arucoParams)

In [ ]:
corners

In [ ]:
plt.imshow(tag)
plt.scatter([276], [605], s = 3)
plt.scatter([866], [587], s = 3)
plt.scatter([852], [1255], s = 3)
plt.scatter([293], [1198], s = 3)
plt.show()

In [ ]:
img_rgb = cv2.imread('/Users/pushkarshinde/Downloads/arukoTags/images/IMG_0781.jpg')

In [ ]:
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

In [ ]:
res = aruco.detectMarkers(img_gray, aruco_dict) ### -> corners, ids, rejectedImgPoints

In [ ]:
corners

In [ ]:
# Draw each marker 
for i in range(len(res[0])):
    ## i->0 as len(res[0]) = 1
    # Estimate pose of the respective marker, with matrix size 1x1
    rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(res[0][i], 1, mtx, dist)
    # Define the ar cube
    # Since we previously set a matrix size of 1x1 for the marker and we want the cube to be the same size, it is also defined with a size of 1x1x1
    # It is important to note that the center of the marker corresponds to the origin and we must therefore move 0.5 away from the origin 
    axis = np.float32([[-0.5, -0.5, 0], [-0.5, 0.5, 0], [0.5, 0.5, 0], [0.5, -0.5, 0],
                       [-0.5, -0.5, 1.0], [-0.5, 0.5, 1.0], [0.5, 0.5, 1.0],[0.5, -0.5, 1.0]])
    # Now we transform the cube to the marker position and project the resulting points into 2d
    imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
    imgpts = np.int32(imgpts).reshape(-1, 2)

    # Now comes the drawing. 
    # In this example, I would like to draw the cube so that the walls also get a painted
    # First create six copies of the original picture (for each side of the cube one)
    side1 = img_rgb.copy()
    side2 = img_rgb.copy()
    side3 = img_rgb.copy()
    side4 = img_rgb.copy()
    side5 = img_rgb.copy()
    side6 = img_rgb.copy()

    # Draw the bottom side (over the marker)
    side1 = cv2.drawContours(side1, [imgpts[:4]], -1, (255, 0, 0), -2)
    # Draw the top side (opposite of the marker)
    side2 = cv2.drawContours(side2, [imgpts[4:]], -1, (255, 0, 0), -2)
    # Draw the right side vertical to the marker
    side3 = cv2.drawContours(side3, [np.array(
        [imgpts[0], imgpts[1], imgpts[5],
         imgpts[4]])], -1, (255, 0, 0), -2)
    # Draw the left side vertical to the marker
    side4 = cv2.drawContours(side4, [np.array(
        [imgpts[2], imgpts[3], imgpts[7],
         imgpts[6]])], -1, (255, 0, 0), -2)
    # Draw the front side vertical to the marker
    side5 = cv2.drawContours(side5, [np.array(
        [imgpts[1], imgpts[2], imgpts[6],
         imgpts[5]])], -1, (255, 0, 0), -2)
    # Draw the back side vertical to the marker
    side6 = cv2.drawContours(side6, [np.array(
        [imgpts[0], imgpts[3], imgpts[7],
         imgpts[4]])], -1, (255, 0, 0), -2)

    # Until here the walls of the cube are drawn in and can be merged
    img_rgb = cv2.addWeighted(side1, 0.1, img_rgb, 0.9, 0)
    img_rgb = cv2.addWeighted(side2, 0.1, img_rgb, 0.9, 0)
    img_rgb = cv2.addWeighted(side3, 0.1, img_rgb, 0.9, 0)
    img_rgb = cv2.addWeighted(side4, 0.1, img_rgb, 0.9, 0)
    img_rgb = cv2.addWeighted(side5, 0.1, img_rgb, 0.9, 0)
    img_rgb = cv2.addWeighted(side6, 0.1, img_rgb, 0.9, 0)

    # Now the edges of the cube are drawn thicker and stronger
    img_rgb = cv2.drawContours(img_rgb, [imgpts[:4]], -1, (255, 0, 0), 2)
    for i, j in zip(range(4), range(4, 8)):
        img_rgb = cv2.line(img_rgb, tuple(imgpts[i]), tuple(imgpts[j]), (255, 0, 0), 2)
    img_rgb = cv2.drawContours(img_rgb, [imgpts[4:]], -1, (255, 0, 0), 2)



In [ ]:
# Display the result
cv2.imshow("AR-Example", cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()